In [21]:
import pandas as pd
import matplotlib as plt

file = "Resources/SpotifyFeatures.csv"
spotify_data = pd.read_csv(file)

In [22]:
spotify_data = spotify_data.loc[(spotify_data["popularity"] >= 50)]
spotify_data

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
135,R&B,Mary J. Blige,Be Without You - Kendu Mix,2YegxR5As7BeQuVp2U6pek,65,0.0830,0.724,246333,0.689,0.000000,D,0.3040,-5.922,Minor,0.1350,146.496,4/4,0.6930
136,R&B,Rihanna,Desperado,6KFaHC9G178beAp7P0Vi5S,63,0.3230,0.685,186467,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,3/4,0.3230
137,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,6muW8cSjJ3rusKJ0vH5olw,62,0.0675,0.762,199520,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4/4,0.0862
138,R&B,Surfaces,Heaven Falls / Fall on Me,7yHqOZfsXYlicyoMt62yC6,61,0.3600,0.563,240597,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4/4,0.7680
139,R&B,Olivia O'Brien,Love Myself,4XzgjxGKqULifVf7mnDIQK,68,0.5960,0.653,213947,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4/4,0.4660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232480,Soul,Lisa Stansfield,So Be It - Soultalk Remix,6KeQciSHzfXjThXqznLSa1,51,0.1240,0.761,251533,0.753,0.000004,B,0.1120,-6.728,Major,0.0325,104.967,4/4,0.8280
232502,Soul,Fat Larry's Band,Act Like You Know,1AOacbO5iSMxrGMihq9QDA,50,0.2400,0.745,390013,0.781,0.000003,F,0.1000,-7.701,Minor,0.1250,103.303,4/4,0.8430
232513,Soul,Paolo Nutini,Rewind,514q6NjO0jX0yVOOpTs2Ez,52,0.1100,0.517,258013,0.422,0.000000,C,0.1100,-9.227,Major,0.0334,87.633,4/4,0.2400
232575,Soul,Joe Henderson,Blue Bossa - Remastered,6qqK0oeBRapZn8f9hJJENw,51,0.9110,0.531,482440,0.366,0.880000,F,0.0916,-11.697,Minor,0.0496,81.867,4/4,0.7850
